In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
head= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.6533.100 Safari/537.36",
  "Accept-Encoding": "gzip, deflate, br",
  "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
  "Connection": "keep-alive",
  "Accept-Language": "en-US,en;q=0.9,lt;q=0.8,et;q=0.7,de;q=0.6"}

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_multiple_imdb_pages(urls):
    # List to store data from each IMDb page
    imdb_data_list = []

    for url in urls:
        # Fetch the HTML content from the URL
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})

        # Check if the request was successful
        if response.status_code == 200:
            content = response.content

            # Parse the HTML content with BeautifulSoup
            soup = BeautifulSoup(content, 'lxml')

            # Extract IMDb details
            try:
                title = soup.find("span", {"class": "hero__primary-text"}).text.strip()
            except AttributeError:
                title = None

            try:
                release_year = soup.find("li", {"class": "ipc-link ipc-link--baseAlt ipc-link--inherit-color"}, {"role":"presentation"}).text.strip()
            except AttributeError:
                release_year = None

            try:
                rating = soup.find("span", {"class": "sc-d541859f-1 imUuxf"}).text.strip()
            except AttributeError:
                rating = None

            try:
                genre_elements = soup.find_all("span", {"class": "ipc-chip__text"})
                genres = [genre.text.strip() for genre in genre_elements]
            except AttributeError:
                genres = None

            try:
                director_element = soup.find("ul", {"class": "ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt"})
                director = director_element.text.strip() if director_element else None
            except AttributeError:
                director = None

            try:
                description = soup.find("span", {"data-testid": "plot-xs_to_m"}).text.strip()
            except AttributeError:
                description = None

            # Append the extracted data to the list
            imdb_details = {
                'Title': title,
                'Release Year': release_year,
                'Rating': rating,
                'Genres': ', '.join(genres) if genres else None,
                'Director': director,
                'Description': description,
                'URL': url
            }
            imdb_data_list.append(imdb_details)
        else:
            print(f"Failed to retrieve data from {url}. Status code: {response.status_code}")

    # Create a DataFrame from the list of IMDb data
    df = pd.DataFrame(imdb_data_list)

    return df

In [ ]:
movie_title = [ "https://www.imdb.com/title/tt0111161/?ref_=chttp_t_1",
             "https://www.imdb.com/title/tt0068646/?ref_=chttp_t_2",
             "https://www.imdb.com/title/tt0468569/?ref_=chttp_t_3" ]

In [ ]:
clean_links = []
for link in movie_title:
  clean_links.append(link.split("?")[0])
print(clean_links)

['https://www.imdb.com/title/tt0111161/', 'https://www.imdb.com/title/tt0068646/', 'https://www.imdb.com/title/tt0468569/']


In [ ]:
df = scrape_multiple_imdb_pages(clean_links)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title         3 non-null      object
 1   Release Year  0 non-null      object
 2   Rating        3 non-null      object
 3   Genres        3 non-null      object
 4   Director      3 non-null      object
 5   Description   3 non-null      object
 6   URL           3 non-null      object
dtypes: object(7)
memory usage: 296.0+ bytes


In [ ]:
df.head()

,Title,Release Year,Rating,Genres,Director,Description,URL
0,The Shawshank Redemption,None,9.3,"Epic, Period Drama, Prison Drama, Drama, Back ...",Frank Darabont,A banker convicted of uxoricide forms a friend...,https://www.imdb.com/title/tt0111161/
1,The Godfather,None,9.2,"Epic, Gangster, Tragedy, Crime, Drama, Back to...",Francis Ford Coppola,The aging patriarch of an organized crime dyna...,https://www.imdb.com/title/tt0068646/
2,The Dark Knight,None,9.0,"Action Epic, Epic, Superhero, Action, Crime, D...",Christopher Nolan,When a menace known as the Joker wreaks havoc ...,https://www.imdb.com/title/tt0468569/
